# Libraries


In [36]:
!pip install --upgrade azureml-core

Requirement already up-to-date: azureml-core in /usr/local/lib/python3.7/dist-packages (1.31.0)


In [1]:
!pip install azureml-dataset-runtime --upgrade

Requirement already up-to-date: azureml-dataset-runtime in /usr/local/lib/python3.7/dist-packages (1.31.0)


In [28]:
!pip install --upgrade azureml-train-automl-client

Requirement already up-to-date: azureml-train-automl-client in /usr/local/lib/python3.7/dist-packages (1.31.0)


In [1]:
!pip install azureml-train-automl

In [1]:
!pip install azureml-pipeline

In [1]:
!pip install azureml-widgets

In [34]:
!python --version

Python 3.7.10


# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [2]:
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'capstone-project-experiment'

experiment = Experiment(ws, experiment_name)

In [4]:
dataset = Dataset.get_by_name(ws, "default-ibk-dataset")
df = dataset.to_pandas_dataframe()

In [5]:
df.head()

,key_value,target,edad,sexo,est_cvl,sit_lab,ctd_hijos,flg_sin_email,ctd_veh,tip_lvledu,total_mean_of_saldo_count_per_month,total_sum_of_saldo_count_per_month,total_min_of_saldo_count_per_month,total_max_of_saldo_count_per_month,total_mean_of_saldo_sum_per_month,total_sum_of_saldo_sum_per_month,total_min_of_saldo_sum_per_month,total_max_of_saldo_sum_per_month,total_min_of_saldo_min_per_month,total_mean_of_saldo_min_per_month,total_max_of_saldo_max_per_month,total_mean_of_saldo_max_per_month,total_min_of_condicion_min_per_month,total_mean_of_condicion_min_per_month,total_max_of_condicion_max_per_month,total_mean_of_condicion_max_per_month,total_mean_of_condicion_mean_per_month
0,0,0,-0.983651,1.0,4.0,2.0,0.0,1.0,0.0,0.0,4.416667,53,3,7,4.245681,50.948174,2.984509,6.829334,0.927141,0.936001,1.072672,1.014399,0,0.0,0,0.000,0.000000
1,1,0,0.000953,1.0,4.0,1.0,0.0,1.0,1.0,0.0,2.666667,8,2,4,2.486586,7.459759,1.864763,3.729381,0.927119,0.927238,0.938372,0.938051,0,0.0,0,0.000,0.000000
2,2,0,0.531124,0.0,4.0,1.0,0.0,1.0,0.0,7.0,3.000000,6,3,3,2.818454,5.636907,2.787291,2.849616,0.927119,0.927119,0.995254,0.964115,0,0.0,0,0.000,0.000000
3,3,0,-1.135129,0.0,4.0,1.0,0.0,1.0,0.0,7.0,3.500000,7,3,4,3.596903,7.193806,3.117342,4.076464,0.927209,0.928015,1.114270,1.104669,0,0.0,0,0.000,0.000000
4,4,1,-1.438084,0.0,4.0,1.0,0.0,1.0,0.0,0.0,7.250000,58,3,12,6.797555,54.380440,2.823033,11.229775,0.927220,0.928755,0.957766,0.952834,0,0.0,80,19.125,1.730303


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [6]:
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException

amlcompute_cluster_name = "capstone-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')

except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 1)

Found existing cluster, use it.
Succeeded............
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


In [7]:
project_folder = './capstone-project-pipeline'

In [8]:
from azureml.train.automl import AutoMLConfig

# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'AUC_weighted',
    "n_cross_validations": 3
    }

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="target",
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [10]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
capstone-project-experiment,AutoML_acb37720-005d-4594-a47d-a4998be64697,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


In [9]:
from azureml.pipeline.core import PipelineData, TrainingOutput

ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))

model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

In [12]:
from azureml.pipeline.core import Pipeline
from azureml.pipeline.steps import AutoMLStep

automl_step = AutoMLStep(name='automl_module', automl_config=automl_config, outputs=[metrics_data, model_data], allow_reuse=True)
pipeline = Pipeline(description="pipeline_with_automlstep", workspace=ws, steps=[automl_step])

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [15]:
%matplotlib inline

In [13]:
from azureml.widgets import RunDetails

In [17]:
#RunDetails(remote_run).show()

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [14]:
best_run, model = remote_run.get_output()

In [19]:
automl_model_name = best_run.properties['model_name']
print('Best AutoML model name: ' + automl_model_name,
      'Best AutoML model run: ' + str(best_run),
      'Best AutoML model specification: ' + str(model), sep = '\n\n')

Best AutoML model name: AutoMLacb37720029

Best AutoML model run: Run(Experiment: capstone-project-experiment,
Id: AutoML_acb37720-005d-4594-a47d-a4998be64697_29,
Type: azureml.scriptrun,
Status: Completed)

Best AutoML model specification: Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/content')),
                ('stackensembleclassifier...
), random_state=None, reg_alpha=0.9473684210526315, reg_lambda=0.42105263157894735, subsample=0.19842105263157894))], verbose=False))], meta_learner=LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False, fit_intercept=True, intercept_scaling=1.0, l1_ratios=None, max_iter=100, multi_class='auto', n_jobs=None, penalty='l2', random_state=

In [18]:
model.steps

[('datatransformer', DataTransformer(
      task='classification',
      is_onnx_compatible=False,
      enable_feature_sweeping=True,
      enable_dnn=False,
      force_text_dnn=False,
      feature_sweeping_timeout=86400,
      featurization_config=None,
      is_cross_validation=True,
      feature_sweeping_config={}
  )), ('stackensembleclassifier', StackEnsembleClassifier(
      base_learners=[('0', Pipeline(memory=None,
               steps=[('maxabsscaler', MaxAbsScaler(copy=True)),
                      ('lightgbmclassifier',
                       LightGBMClassifier(min_data_in_leaf=20, n_jobs=1, problem_info=ProblemInfo(
          dataset_samples=354663,
          dataset_features=81,
          dataset_classes=2,
          dataset_num_categorical=0,
          dataset_categoricals=None,
          pipeline_categoricals=None,
          dataset_y_std=None,
          dataset_uid=None,
          subsamplin...
          subsampling_schedule='hyperband_clip',
          cost_mode_par

In [20]:
#TODO: Save the best model

# Register and save the best model
automl_model_registered = remote_run.register_model(model_name='default_automl_model')

automl_model_registered.download(target_dir="outputs_automl", exist_ok=True)

'outputs_automl/model.pkl'

In [23]:
!ls -l outputs_automl

total 4644
-rw-r--r-- 1 root root 4752705 Jun 29 19:46 model.pkl


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [30]:
from azureml.core.webservice import AciWebservice
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.model import Model

In [35]:
myenv = Environment(name="capstoneenv")
myenv.register(workspace=ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210531.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "capstoneenv",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge"
  

In [36]:
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, 
                                                memory_gb=4, 
                                                enable_app_insights=True,
                                                description='Predict default score')

model = Model(ws, 'default_automl_model')

env_deploy = Environment.get(workspace=ws, name='capstoneenv')

inference_config = InferenceConfig(entry_script="score.py", environment=env_deploy)

service = Model.deploy(workspace=ws, 
                       name='default-automl', 
                       models=[model], 
                       inference_config=inference_config, 
                       deployment_config=aci_config)

In [37]:
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-06-29 20:15:33+00:00 Creating Container Registry if not exists.
2021-06-29 20:15:33+00:00 Registering the environment.
2021-06-29 20:15:34+00:00 Building image..
2021-06-29 20:19:49+00:00 Generating deployment configuration.
2021-06-29 20:19:50+00:00 Submitting deployment to compute..
2021-06-29 20:19:53+00:00 Checking the status of deployment default-automl..
2021-06-29 20:21:31+00:00 Checking the status of inference endpoint default-automl.
Failed


ERROR:azureml.core.webservice.webservice:Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: cfa290eb-37e6-4b00-804a-4a959a50c4b6
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: default-automl. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image azudacitycr01.azurecr.io/azureml/azureml_2dc4016849bfdcec9f9d4bb2200f06f4 locally. Please refer to https://aka.ms/debugim

WebserviceException: ignored

In [ ]:
print(service.scoring_uri)
print(service.swagger_uri)

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service